In [0]:
!pip install tensorflow==1.14.0

In [0]:


import os
from google.colab import drive
drive.mount('/content/drive')

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"
# COMP_PATH = 'C:/Users/moong/Desktop/CS5242 Project/'
# COMP_PATH = 'C:\\Users\\Calvin\\Documents\\Python Scripts\\NUS\\CS5242 Neural Networks and Deep Learning\\Project'

os.chdir(COMP_PATH)
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['splits',
 'groundTruth',
 'airplane.png',
 'segment.txt',
 'main.py',
 'data',
 '__pycache__',
 'Project CS5242 - Data Exploration.ipynb',
 'model.png',
 'model.json',
 'model.h5',
 'Load Model',
 'All mains',
 'All outputs',
 'Copy of read_datasetBreakfast.py',
 'read_datasetBreakfast.py',
 'start_run.ipynb',
 'Tuning.h5',
 'Tuning.LSTM.h5',
 'tuning_LSTM.output.txt',
 'Tuning_fit_gen.CNNLSTM.cmd.py',
 'Tuning_fit_gen.CNNLSTM.cmd.txt',
 'Tuning_fit_gen.LSTM.cmd.py',
 'Tuning_fit_gen.main.ipynb',
 'loadfile_output.txt',
 'start_run - 2.ipynb',
 'Copy of LSTM Main.ipynb',
 'LSTM Main.ipynb',
 'Copy of start_run - 2.ipynb']

In [0]:
# import main
# from read_datasetBreakfast import load_data, read_mapping_dict
from read_datasetBreakfast import read_mapping_dict, load_data, get_label_bounds
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
import torch

from keras.utils import np_utils
from keras.callbacks import History 

from datetime import datetime

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:

COMP_PATH = "/content/drive/My Drive/Colab Notebooks/NUS/CS5242 Neural Networks and Deep Learning/Project"

''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
# train_split =  os.path.join(COMP_PATH, 'splits/2 train.split1.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/2 valid.split1.bundle') #Train Split 
# train_split =  os.path.join(COMP_PATH, 'splits/300 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/90 valid.split1 - P49_54.bundle') # Valid Split 
# train_split =  os.path.join(COMP_PATH, 'splits/900 train.split1 - P16_48.bundle') #Train Split
# valid_split =  os.path.join(COMP_PATH, 'splits/150 valid.split1 - P49_54.bundle') # Valid Split 
train_split =  os.path.join(COMP_PATH, 'splits/train.split1 - Orig.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)
if  split == 'training':
    startTime = datetime.now()
    data_feat, data_labels = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
    labels_uniq, labels_uniq_loc = get_label_bounds(data_labels)
    endTime = datetime.now()
    print("******** Total Time to load file:",(endTime - startTime))
if  split == 'test':
    data_feat = load_data( test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only

# print(len(data_feat))
# print(len(data_labels))
# print(data_feat)
# print(data_labels)

Finish Load the Training data and labels!!!
******** Total Time to load file: 0:39:31.804065


In [0]:
def add_video_segment(video_segment, segment_label, all_video_segments, all_video_labels, target_len):
    video_len = len(video_segment)
    if (video_len > 0) and (segment_label != 0):
        if (video_len < target_len):
            padded_video_segment = np.zeros((target_len, 400))
            padded_video_segment[0:len(video_segment)] = video_segment
        elif (video_len > target_len):
            # Get the middle part
            start_idx = (video_len-target_len)//2
            padded_video_segment = video_segment[start_idx:start_idx+target_len]
        else:
            padded_video_segment = video_segment
        all_video_segments.append(padded_video_segment)
        all_video_labels.append(segment_label)
    return all_video_segments, all_video_labels

In [0]:
X_data = [data_feat[i].cpu().detach().numpy() for i in range(len(data_feat))]
Y_data = data_labels

target_len = 50

all_video_segments = []
all_video_labels = []
for i, label_group in enumerate(Y_data):
    video_group = X_data[i]
    video_segment = []
    segment_label = label_group[0]
    
    idx = 0
    for j, label in enumerate(label_group):
        
        if (label != segment_label):
            all_video_segments, all_video_labels = add_video_segment(
                video_segment, segment_label, all_video_segments, all_video_labels, target_len)
            segment_label = label
            video_segment = []
            idx = 0
        video_segment.append(video_group[j])
        idx += 1
        if idx == target_len:
            all_video_segments, all_video_labels = add_video_segment(
                video_segment, segment_label, all_video_segments, all_video_labels, target_len)
            video_segment = []
            idx = 0

    all_video_segments, all_video_labels = add_video_segment(
        video_segment, segment_label, all_video_segments, all_video_labels, target_len)
        
X = np.array(all_video_segments)
Y = np.array(all_video_labels)

print(X.shape)
print(Y.shape)
    

In [0]:
Y = np_utils.to_categorical(Y, num_classes=48)
print(Y.shape)


In [0]:
n_hidden_nodes = 30
n_dense_nodes = 40
n_features = 400
n_timesteps = target_len
n_outputs = 48

n_epoch = 100
batch_size = 64
init_lr = 0.01

clipnorm=1.0
clipvalue=0.5

In [0]:
from keras import optimizers
from keras import regularizers

# from keras.models import Model, Sequential
# from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
import tensorflow as tf


lr = tf.compat.v1.train.exponential_decay(
    init_lr,                # Base learning rate.
    batch_size,  # Current index into the dataset.
    8000,          # Decay step.
    0.9,                # Decay rate.
    staircase=True)

adam = optimizers.Adam(lr=lr, clipnorm=clipnorm, clipvalue=clipvalue)



model = Sequential()
# model.add(Bidirectional(LSTM(n_hidden_nodes, 
#                              input_shape=(n_timesteps, n_features), 
#                              kernel_initializer='random_uniform',
#                              recurrent_initializer='random_uniform',
#                             #  kernel_regularizer=regularizers.l2(0.01), 
#                             #  recurrent_regularizer=regularizers.l2(0.01) 
#                              )))
# model.add(Bidirectional(LSTM(n_hidden_nodes, input_shape=(n_timesteps, n_features))))
model.add(Bidirectional(LSTM(n_hidden_nodes, return_sequences=True, input_shape=(n_timesteps, n_features))))
# model.add(Dropout(0.2))
model.add(LSTM(20))
# model.add(Dropout(0.2))
model.add(Dense(n_dense_nodes, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(200, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

# fit model
# print(all_padded_video_segments)
# print(all_padded_video_labels)

# Split train vs valid data set
ratio = 0.9
x_train_size = int(X.shape[0]*ratio)
x_train = X[:x_train_size]
y_train = Y[:x_train_size]
x_valid = X[x_train_size:Y.shape[0]]
y_valid = Y[x_train_size:Y.shape[0]]

print("x_train size: ", x_train.shape[0])
print("x_valid size: ", x_valid.shape[0])


history = History()

startTime = datetime.now()
# for i in range(n_epoch):
history = model.fit(X, Y, epochs=n_epoch, batch_size=batch_size, validation_split=0.1, verbose=2, callbacks=[history])
endTime = datetime.now()
print("*** Time took: "+str(endTime-startTime))
    # print(history)
#     print('acc %.2f, loss %.2f' % (acc, loss), end='')

# for i, video_segment in enumerate(all_padded_video_segments):
#     video_segment = np.array(video_segment).reshape(1,n_timesteps,n_features)
#     video_label = np.array(one_hot_all_video_labels[i])
#     print(video_label.shape)
#     model.fit(video_segment, video_label, epochs=1, verbose=1, shuffle=False)
    
print('\n# Evaluate on test data')
results = model.evaluate(x_valid, y_valid)
print('test loss, test acc:', results)

In [0]:
a = []
while True:
    a.append("1")